In [1]:
import pandas as pd
filepath = filepath = 'C:/Users/Wai/OneDrive/Desktop/insurance/'
df = pd.read_csv(filepath + 'Super.csv')

In [2]:
%matplotlib inline

In [3]:
df1 = df.filter(['Ship Mode','Segment','Region','Category','Sub-Category','Profit'], axis=1)
df1.head()

,Ship Mode,Segment,Region,Category,Sub-Category,Profit
0,Second Class,Consumer,South,Furniture,Bookcases,41.9136
1,Second Class,Consumer,South,Furniture,Chairs,219.5820
2,Second Class,Corporate,West,Office Supplies,Labels,6.8714
3,Standard Class,Consumer,South,Furniture,Tables,-383.0310
4,Standard Class,Consumer,South,Office Supplies,Storage,2.5164


In [4]:
df1.shape

(9994, 6)

In [5]:
df2=df1.groupby(['Region', 'Segment'], as_index=False)['Profit'].count()
df2.head()

,Region,Segment,Profit
0,Central,Consumer,1212
1,Central,Corporate,673
2,Central,Home Office,438
3,East,Consumer,1469
4,East,Corporate,877


In [6]:
df3=df1.groupby(['Segment', 'Category'], as_index=False)['Profit'].count()
df3.head()

,Segment,Category,Profit
0,Consumer,Furniture,1113
1,Consumer,Office Supplies,3127
2,Consumer,Technology,951
3,Corporate,Furniture,646
4,Corporate,Office Supplies,1820


In [7]:
df4=df1.groupby(['Category','Sub-Category'], as_index=False)['Profit'].count()
df4.head()

,Category,Sub-Category,Profit
0,Furniture,Bookcases,228
1,Furniture,Chairs,617
2,Furniture,Furnishings,957
3,Furniture,Tables,319
4,Office Supplies,Appliances,466


In [8]:
df5=df1.groupby(['Sub-Category','Ship Mode'], as_index=False)['Profit'].count()
df5.head()

,Sub-Category,Ship Mode,Profit
0,Accessories,First Class,128
1,Accessories,Same Day,41
2,Accessories,Second Class,162
3,Accessories,Standard Class,444
4,Appliances,First Class,76


In [9]:
df2.columns = ['a', 'b', 'Quantity']
df2.head()

,a,b,Quantity
0,Central,Consumer,1212
1,Central,Corporate,673
2,Central,Home Office,438
3,East,Consumer,1469
4,East,Corporate,877


In [10]:
df3.columns = ['a', 'b', 'Quantity']
df4.columns = ['a', 'b', 'Quantity']
df5.columns = ['a', 'b', 'Quantity']

In [11]:
df6 = df2.append(df3)
df6.head()

,a,b,Quantity
0,Central,Consumer,1212
1,Central,Corporate,673
2,Central,Home Office,438
3,East,Consumer,1469
4,East,Corporate,877


In [12]:
df7 = df6.append(df4)
df8 = df7.append(df5)
df8

,a,b,Quantity
0,Central,Consumer,1212
1,Central,Corporate,673
2,Central,Home Office,438
3,East,Consumer,1469
4,East,Corporate,877
...,...,...,...
63,Supplies,Standard Class,111
64,Tables,First Class,47
65,Tables,Same Day,21
66,Tables,Second Class,61


In [13]:
df8.to_csv(filepath + 'Sankey_data.csv')
#import plotly
#import chart_studio
#import chart_studio.plotly.plotly.offline as py

In [14]:
import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.offline as py

In [15]:
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#FFD43B','#646464','#4B8BBE','#306998']
    
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig

In [16]:
fig = genSankey(df8,cat_cols=['a','b'],value_cols='Quantity',title='Sankey Diagram x Profit')

In [17]:
py.plot(fig, validate=False)

'temp-plot.html'

### Cluster > DOW > Hour > Open - Clicked > Keywords > ANB / VONB